In [16]:
import cv2
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy
from sklearn.preprocessing import StandardScaler    
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

# https://www.cs.virginia.edu/~vicente/recognition/notebooks/image_processing_lab.html
# and Yankun 
# Write the function for uploading all png files in the file folder.
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder,filename))
        if img is not None:
            images.append(img)
    return images

folder = r"C:\Users\hands\Fiolic\images46"

#https://pytorch.org/tutorials/beginner/pytorch_with_examples.html

In [6]:
#load all the images , right now it was a folder of the first 46 images from images_001 
img = load_images_from_folder(folder)
print(len(img))

46


In [7]:
#convert the ima to a FloatTensor object
img_tensor = torch.FloatTensor(img)
print(img_tensor.shape)

torch.Size([46, 1024, 1024, 3])


In [8]:
#choose first 40 for training
#choose last 6 for testing
x_train = img_tensor[:40]
x_test = img_tensor[40:]
print(x_train.shape)
print(x_test.shape)


torch.Size([40, 1024, 1024, 3])
torch.Size([6, 1024, 1024, 3])


In [9]:
#I used the first 2 colums of the Data_entry_2017.csv in a new csv called testdata

with open("testdata.csv", "r") as t:
    x = t.readlines() 

In [10]:
#import the data into a list, for lables 
#if the image had "No Finding" then a 0 was recorded, anything else was a 1 
imgLabels = []
for line in x:
    if "No Finding" in line:
        imgLabels.append([0])
    else:
        imgLabels.append([1])
        
print(imgLabels)

[[1], [1], [1], [0], [1], [1], [1], [1], [1], [1], [1], [1], [1], [0], [0], [0], [0], [0], [0], [1], [1], [0], [0], [1], [0], [1], [1], [1], [1], [0], [0], [0], [0], [1], [1], [1], [0], [1], [0], [1], [1], [1], [1], [1], [1], [1]]


In [11]:
#same as before, now for labels----- turn to tensor float then split into 
labels_tensor = torch.FloatTensor(imgLabels)
y_train = labels_tensor[:40]
y_test = labels_tensor[40:]

print(y_train.shape)
print(y_test.shape)

torch.Size([40, 1])
torch.Size([6, 1])


In [12]:
#https://towardsdatascience.com/pytorch-tabular-binary-classification-a0368da5bb89
EPOCHS = 20
BATCH_SIZE = 1
LEARNING_RATE = 0.001

class trainData(Dataset):
    
    def __init__(self, X_data, y_data):
        self.X_data = X_data
        self.y_data = y_data
        
    def __getitem__(self, index):
        return self.X_data[index], self.y_data[index]
        
    def __len__ (self):
        return len(self.X_data)


train_data = trainData(x_train, 
                       y_train)
## test data    
class testData(Dataset):
    
    def __init__(self, X_data):
        self.X_data = X_data
        
    def __getitem__(self, index):
        return self.X_data[index]
        
    def __len__ (self):
        return len(self.X_data)
    

test_data = testData(x_test)

In [13]:
#print(train_data[0])
#combine the data for images and lables,
# make into DataLoader object
train_loader = DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(dataset=test_data, batch_size=1)

In [14]:
https://towardsdatascience.com/training-a-neural-network-using-pytorch-72ab708da210
model = nn.Sequential(nn.Linear(3145728, 256),
                      nn.ReLU(),
                      nn.Linear(256, 10),
                      nn.LogSoftmax(dim=1))
# Define the loss
criterion = nn.NLLLoss()
# Optimizers require the parameters to optimize and a learning rate
optimizer = optim.SGD(model.parameters(), lr=0.003)
epochs = 5
for e in range(epochs):
    running_loss = 0
    for images, labels in train_loader:
#         print(images.shape)
#         print(labels.shape)
        # Flatten MNIST images into long vector
        
        images = images.view(images.shape[0], -1)
#         print(images)
        # Training pass
        optimizer.zero_grad()
        
        output = model(images)
        loss = criterion(output, labels[0].type(torch.LongTensor))
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    else:
        print(f"Training loss: {running_loss/len(train_loader)}")

Training loss: 149461618463.26093
Training loss: 607773095.4190052
Training loss: 2.196513956785202
Training loss: 2.149394851922989
Training loss: 2.103535979986191


In [18]:
#figure out if GPU can be used
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [20]:
#https://towardsdatascience.com/pytorch-tabular-binary-classification-a0368da5bb89
y_pred_list = []
model.eval()
with torch.no_grad():
    for X_batch in test_loader:
        X_batch = X_batch.to(device)
        y_test_pred = model(X_batch.view(X_batch.shape[0], -1))
        y_test_pred = torch.sigmoid(y_test_pred)
        y_pred_tag = torch.round(y_test_pred)
        y_pred_list.append(y_pred_tag.cpu().numpy())

y_pred_list = [a.squeeze().tolist() for a in y_pred_list]

In [26]:

#confusion_matrix(y_test, y_pred_list)

In [25]:
#print(classification_report(y_test, y_pred_list))